In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
import wandb
import os

In [ ]:
from rdkit import rdBase
blocker = rdBase.BlockLogs()

In [ ]:
!python3 -m wandb login $WANDB_TOKEN

### Upload config

In [ ]:
import yaml

config = yaml.load(open("config-qsar-classification-exact.yaml", "r"), Loader=yaml.FullLoader)
print(config)

In [ ]:
print('batch_size =', config['batch_size'])

In [ ]:
print('running on device:', config['gpu'])
device = torch.device(config['gpu']) if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def _save_config_file(config, log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    with open(os.path.join(log_dir, 'config.yml'), 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False, sort_keys=False)

### Upload and Split Dataset

In [ ]:
dataframe = pd.concat([pd.read_csv("BBBP_train.csv"), pd.read_csv("BBBP_test.csv"),  pd.read_csv("BBBP_valid.csv")])

In [ ]:
df1, df2, df3 = pd.read_csv("BBBP_train.csv"), pd.read_csv("BBBP_test.csv"),  pd.read_csv("BBBP_valid.csv")
train_idx = df1['num']
test_idx = df2['num']
val_idx = df3['num']

In [ ]:
dataframe

In [ ]:
import ast
tqdm.pandas()

def string_to_array(input_string):
    try:
        # Use ast.literal_eval to safely evaluate the string as a Python literal
        result = ast.literal_eval(input_string)
        return result
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the string: {e}")
        return None

dataframe['descriptors'] = dataframe['descriptors'].progress_apply(lambda s: string_to_array(s))

In [ ]:
from rdkit import Chem


dropping = []
for i in tqdm(range(len(dataframe['smiles']))):
    mol = Chem.MolFromSmiles(dataframe['smiles'].iloc[i])
    if mol is None:
        dropping.append(i)
        continue
    
    if mol.GetNumAtoms() < 2:
        dropping.append(i)

dataframe = dataframe.drop(dropping)
dataframe.reset_index(inplace = True)

In [ ]:
train_idx = [x for x in train_idx if x in dataframe['num']]
test_idx = [x for x in test_idx if x in dataframe['num']]
val_idx = [x for x in val_idx if x in dataframe['num']]

In [ ]:
dataframe

### Create Molecule Dataset
##### It will generate torch_geometric.data.Data objects for both bert and GIN/GCN models.

In [ ]:
from rdkit import Chem

ATOM_LIST = list(range(1,119))
CHIRALITY_LIST = [
    Chem.rdchem.ChiralType.CHI_UNSPECIFIED,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
    Chem.rdchem.ChiralType.CHI_OTHER
]
BOND_LIST = [
    Chem.rdchem.BondType.SINGLE, 
    Chem.rdchem.BondType.DOUBLE, 
    Chem.rdchem.BondType.TRIPLE, 
    Chem.rdchem.BondType.AROMATIC
]
BONDDIR_LIST = [
    Chem.rdchem.BondDir.NONE,
    Chem.rdchem.BondDir.ENDUPRIGHT,
    Chem.rdchem.BondDir.ENDDOWNRIGHT
]

In [ ]:
tqdm.pandas()

In [ ]:
import random
import math
from copy import deepcopy
from torch_geometric.data import Data, Dataset
import shifter as sh
shifter = sh.Shifter()
import tokenizer as tokenizer


class MoleculeDataset(Dataset):
    def __init__(self, dataset: pd.DataFrame, node_mask_percent=0.15, edge_mask_percent=0.25):
        super(Dataset, self).__init__()
        self.dataset = dataset
        self.node_mask_percent = node_mask_percent
        self.edge_mask_percent = edge_mask_percent
        self.input_ids = []
        self.mask = []
        self.labels = []
        

        self.dataset['graph'] = self.dataset['smiles'].progress_apply(self.get_graph_from_smiles)
        self.dataset['graph_copy1'] = self.dataset['graph'].progress_apply(lambda x: self.get_augmented_graph_copy(x[0], x[1], x[2], x[3], x[4]))
        self.dataset['graph_copy2'] = self.dataset['graph'].progress_apply(lambda x: self.get_augmented_graph_copy(x[0], x[1], x[2], x[3], x[4]))


        for descriptors_of_substructures in self.dataset['descriptors']:
            shifter.shift(descriptors_of_substructures)

        self.maximum = 0

        for mol in self.dataset['descriptors']:
            for substr in mol:
                if substr == '$':
                    continue
                for descriptor in substr:
                    for i in descriptor:
                        self.maximum = max(self.maximum, i)
        
        self.tokenize_descriptors(self.dataset)
        l =[]
        inp = []
        msk = []
        for i in range(len(self.labels)):
            l.append(self.labels[i])
            inp.append(self.input_ids[i])
            msk.append(self.mask[i])

        self.dataset['labels'] = l
        self.dataset['input_ids'] = inp
        self.dataset['attention_mask'] = msk
        
        self.dataset['mlm'] = self.dataset.progress_apply(self.apply_mlm, axis=1)

    def get_graph_from_smiles(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return torch.tensor([[], []], dtype=torch.long), \
                    torch.tensor(np.array([]), dtype=torch.long), \
                    torch.tensor(np.array([]), dtype=torch.long), \
                    0
    
        N = mol.GetNumAtoms()
        M = mol.GetNumBonds()
    
        type_idx = []
        chirality_idx = []
        atomic_number = []
        
        for atom in mol.GetAtoms():
            type_idx.append(ATOM_LIST.index(atom.GetAtomicNum()))
            chirality_idx.append(CHIRALITY_LIST.index(atom.GetChiralTag()))
            atomic_number.append(atom.GetAtomicNum())
        
        x1 = torch.tensor(type_idx, dtype=torch.long).view(-1,1)
        x2 = torch.tensor(chirality_idx, dtype=torch.long).view(-1,1)
        node_feat = torch.cat([x1, x2], dim=-1)
    
        row, col, edge_feat = [], [], []
        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            row += [start, end]
            col += [end, start]
            
            edge_feat.append([
                BOND_LIST.index(bond.GetBondType()),
                BONDDIR_LIST.index(bond.GetBondDir())
            ])
            edge_feat.append([
                BOND_LIST.index(bond.GetBondType()),
                BONDDIR_LIST.index(bond.GetBondDir())
            ])
    
        edge_index = torch.tensor([row, col], dtype=torch.long)
        edge_attr = torch.tensor(edge_feat, dtype=torch.long)
        num_nodes = N
        num_edges = M
        return node_feat, edge_index, edge_attr, num_nodes, num_edges

    def get_augmented_graph_copy(self, node_feat, edge_index, edge_attr, N, M):
        num_mask_nodes = max([1, math.floor(self.node_mask_percent * N)])

        mask_nodes = random.sample(list(range(N)), num_mask_nodes)

        node_feat_new = deepcopy(node_feat)
        for atom_idx in mask_nodes:
            node_feat_new[atom_idx, :] = torch.tensor([len(ATOM_LIST), 0])
        return Data(x=node_feat_new, edge_index=edge_index, edge_attr=edge_attr)

    def tokenize(self, item):
        return self.tokenizer(item, truncation=True, max_length=512, padding='max_length')

    def tokenize_descriptors(self, data):
        sample = tokenizer.tokenize(data['descriptors'], max_length=512)
        
        self.labels.append(torch.tensor(sample['input_ids']))
        self.mask.append(torch.tensor(sample['attention_mask']))
        self.input_ids.append(self.mlm(self.labels[-1].detach().clone()))
        
        self.input_ids = torch.cat(self.input_ids)
        self.mask = torch.cat(self.mask)
        
        self.labels = torch.cat(self.labels)

    def mlm(self, tensor):
        rand = torch.rand(tensor.shape)
        mask_arr = (rand < .15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
        for i in range(tensor.shape[0]):
            selection = torch.flatten(mask_arr[i].nonzero()).tolist()
            tensor[i, selection] = 4
        return tensor

    def apply_mlm(self, sample):
        labels = torch.tensor(sample.input_ids)
        attention_mask = torch.tensor(sample.attention_mask)
        input_ids = torch.tensor(sample.input_ids)
        return Data(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    def __getitem__(self, index):
        return self.dataset['mlm'].iloc[index], self.dataset['graph_copy1'].iloc[index], self.dataset['graph_copy2'].iloc[index], int(self.dataset['p_np'].iloc[index])

    def __len__(self):
        return len(self.dataset)

    def get(self):
        pass
    def len(self):
        pass

In [ ]:
dataset = MoleculeDataset(dataframe)

In [ ]:
from torch_geometric.loader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


num_train = len(dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
    
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(val_idx)
test_sampler = SubsetRandomSampler(test_idx)
    
    
train_dataloader = DataLoader(
    dataset, batch_size=config['batch_size'], sampler=train_sampler,
    num_workers=config['dataset']['num_workers'], drop_last=True
)
    
eval_dataloader = DataLoader(
    dataset, batch_size=config['batch_size'], sampler=valid_sampler,
    num_workers=config['dataset']['num_workers'], drop_last=True
)
    
test_dataloader = DataLoader(
    dataset, batch_size=config['batch_size'], sampler=test_sampler,
    num_workers=config['dataset']['num_workers'], drop_last=True
)

### Create contrastive loss

In [ ]:
import torch
import numpy as np


class NTXentLoss(torch.nn.Module):

    def __init__(self, device, batch_size, temperature, use_cosine_similarity):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.device = device
        self.softmax = torch.nn.Softmax(dim=-1)
        self.mask_samples_from_same_repr = self._get_correlated_mask().type(torch.bool)
        self.similarity_function = self._get_similarity_function(use_cosine_similarity)
        self.criterion = torch.nn.CrossEntropyLoss(reduction="sum")

    def _get_similarity_function(self, use_cosine_similarity):
        if use_cosine_similarity:
            self._cosine_similarity = torch.nn.CosineSimilarity(dim=-1)
            return self._cosine_simililarity
        else:
            return self._dot_simililarity

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = torch.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(torch.bool)
        return mask.to(self.device)

    @staticmethod
    def _dot_simililarity(x, y):
        v = torch.tensordot(x.unsqueeze(1), y.T.unsqueeze(0), dims=2)
        # x shape: (N, 1, C)
        # y shape: (1, C, 2N)
        # v shape: (N, 2N)
        return v

    def _cosine_simililarity(self, x, y):
        # x shape: (N, 1, C)
        # y shape: (1, 2N, C)
        # v shape: (N, 2N)
        v = self._cosine_similarity(x.unsqueeze(1), y.unsqueeze(0))
        return v

    def forward(self, zis, zjs):
        representations = torch.cat([zjs, zis], dim=0)

        similarity_matrix = self.similarity_function(representations, representations)

        # filter out the scores from the positive samples
        l_pos = torch.diag(similarity_matrix, self.batch_size)
        r_pos = torch.diag(similarity_matrix, -self.batch_size)
        positives = torch.cat([l_pos, r_pos]).view(2 * self.batch_size, 1)
        negatives = similarity_matrix[self.mask_samples_from_same_repr].view(2 * self.batch_size, -1)

        logits = torch.cat((positives, negatives), dim=1)
        logits = logits.abs() + 0.0001
        logits = torch.log(logits)
        logits /= self.temperature
        
        labels = torch.zeros(2 * self.batch_size).to(self.device).long()
        loss = self.criterion(logits, labels)

        return loss / (2 * self.batch_size)

### Create Transformer Model

In [ ]:
from transformers import RobertaForMaskedLM
from transformers import RobertaConfig
from torch import nn

if config['graph_model_type'] == 'gin':
    from graph_models.models.ginet_old import GINet as GraphModel
elif config['graph_model_type'] == 'gcn':
    from graph_models.models.gcn import GCN as GraphModel
else:
    raise ValueError('GNN model is not defined in config.')

class MolecularBertGraph(torch.nn.Module):
    def __init__(self):
        super(MolecularBertGraph, self).__init__()
        self.batch_size = config['batch_size']

        roberta_config = roberta_config = RobertaConfig(
            vocab_size=30_522,
            max_position_embeddings=514,
            hidden_size=768,
            num_attention_heads=12,
            num_hidden_layers=6,
            type_vocab_size=1
        )
        
        self.bert = RobertaForMaskedLM(roberta_config).to(device)

        self.graph_model = GraphModel(**config['graph_model']).to(device)
        # self.graph_model = self._load_graph_pretrained_weights(self.graph_model)

        self.out_graph_linear = torch.nn.Linear(2 * config['graph_model']['feat_dim'], 
                                                768, bias=True)

        self.out_graph_projection1 = torch.nn.Linear(768, 768, bias=True)

        self.bn1_graph = nn.BatchNorm1d(768)

        self.out_graph_projection2 = torch.nn.Linear(768, 768, bias=True)

        self.bn2_graph = nn.BatchNorm1d(768)

        self.out_bert_projection1 = torch.nn.Linear(768, 768, bias=True)

        self.bn1_bert = nn.BatchNorm1d(768)

        self.out_bert_projection2 = torch.nn.Linear(768, 768, bias=True)
        
        self.bn2_bert = nn.BatchNorm1d(768)
        
        # contrastive loss
        self.nt_xent_criterion = NTXentLoss(device, self.batch_size, **config['ntxent_loss'])

    def forward(self, bert_batch, graph_batch1, graph_batch2):
        bert_output = self.bert(input_ids=bert_batch['input_ids'].view(self.batch_size, -1), 
                                 attention_mask=bert_batch['attention_mask'].view(self.batch_size, -1),
                                 labels=bert_batch['labels'].view(self.batch_size, -1), output_hidden_states=True)
        bert_loss = bert_output.loss
        bert_emb = bert_output.hidden_states[0][:, 0, :] # take emb for CLS token

        graph_loss, hidden_states_1, hidden_states_2 = self.graph_step(graph_batch1, graph_batch2)
        #graph_loss.backward()
        
        graph_emb = self.out_graph_linear(torch.cat((hidden_states_1, hidden_states_2), dim=-1))

        graph_emb_projected1 = self.out_graph_projection1(graph_emb)
        
        graph_emb_projected_bn1 = self.bn1_graph(graph_emb_projected1)

        graph_emb_projected2 = self.out_graph_projection2(torch.nn.functional.relu(graph_emb_projected_bn1))
        
        graph_emb_projected_bn2 = self.bn2_graph(graph_emb_projected2)
        
        #bert projections:
        bert_emb_projected1 = self.out_bert_projection1(bert_emb)
        
        bert_emb_projected_bn1 = self.bn1_bert(bert_emb_projected1)

        bert_emb_projected2 = self.out_bert_projection2(torch.nn.functional.relu(bert_emb_projected_bn1))
        
        bert_emb_projected_bn2 = self.bn2_bert(bert_emb_projected2)

        bimodal_loss = self.nt_xent_criterion(bert_emb_projected_bn2, graph_emb_projected_bn2)
        return bert_loss, graph_loss, bimodal_loss, graph_emb_projected_bn2, bert_emb_projected_bn2

    def graph_step(self, xis, xjs):
        # get the representations and the projections
        ris, zis = self.graph_model(xis)  # [N,C]
    
        # get the representations and the projections
        rjs, zjs = self.graph_model(xjs)  # [N,C]
    
        # normalize projection feature vectors
        zis = torch.nn.functional.normalize(zis, dim=1)
        zjs = torch.nn.functional.normalize(zjs, dim=1)

        loss = self.nt_xent_criterion(zis, zjs)
        return loss, ris, rjs
        
    def _load_graph_pretrained_weights(self, model):
        try:
            checkpoints_folder = os.path.join('graph_models', 'ckpt', config['load_graph_model'], 'checkpoints')
            print(os.path.join(checkpoints_folder, 'model.pth'))
            state_dict = torch.load(os.path.join(checkpoints_folder, 'model.pth'))
            
            model.load_state_dict(state_dict)
            print("Loaded pre-trained model with success.")
        except FileNotFoundError:
            print("Pre-trained weights not found. Training from scratch.")

        return model

### Define utils

In [ ]:
wandb.init(
    project="bert_bimodal",
    name="BBBP QSAR-all metrics-gin",
    config=config
)

### Training (with validation)

In [ ]:
epoch_counter = 0

In [ ]:
from datetime import datetime

model_checkpoints_folder = os.path.join('ckpts')
dir_name = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(model_checkpoints_folder, dir_name)
_save_config_file(config, log_dir)

In [ ]:
class MolecularPropertiesClassification(torch.nn.Module):
    def __init__(self):
        super(MolecularPropertiesClassification, self).__init__()

        self.test = MolecularBertGraph().to(device)
        self.test.load_state_dict(torch.load('gin_9.pth'))

        self.linear1 = torch.nn.Linear(768 * 2, 768, bias=True)
        self.linear2 = torch.nn.Linear(768, 2, bias=True)

    def forward(self, b, g1, g2):
        l1, l2, l3, graph_emb, bert_emb = self.test(b, g1, g2)
        
        first_linear_out = self.linear1(torch.cat((graph_emb, bert_emb), dim=-1))
        logits = self.linear2(torch.nn.functional.sigmoid(first_linear_out))

        return logits

In [ ]:
model = MolecularPropertiesClassification().to(device)

In [ ]:
model

In [ ]:
num_epoch = config['epochs']

optimizer = torch.optim.Adam(
    model.parameters(), float(config['init_lr']), 
    weight_decay=eval(config['weight_decay'])
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=config['epochs']-config['warm_up'], 
    eta_min=0, last_epoch=-1
)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
def train_loop():
    train_tqdm = tqdm(train_dataloader, unit="batch")
    train_tqdm.set_description(f'Epoch {epoch_counter}')
    loss_sum = 0
    total_pred_labels = []
    total_true_labels = []
    
    model.train()
    for (bert_batch, graph_batch1, graph_batch2, targets) in train_tqdm:
        optimizer.zero_grad()

        bert_batch = bert_batch.to(device)
        graph_batch1 = graph_batch1.to(device)
        graph_batch2 = graph_batch2.to(device)
        targets = targets.clone().detach().to(device)

        logits = model(bert_batch, graph_batch1, graph_batch2)

        loss = loss_func(logits.view(-1, 2), targets.view(-1))
        loss.backward()

        pred_labels = torch.argmax(logits, dim=-1)
        true_labels = targets
        total_pred_labels.append(pred_labels)
        total_true_labels.append(true_labels)
        
        loss_sum += loss.item()

        optimizer.step()
        train_tqdm.set_postfix(loss=loss.item())
    return loss_sum / len(train_dataloader), total_pred_labels, total_true_labels

In [ ]:
def eval_loop():
    eval_tqdm = tqdm(eval_dataloader, unit="batch")
    eval_tqdm.set_description(f'Epoch {epoch_counter}')
    loss_sum = 0
    total_pred_labels = []
    total_true_labels = []
    
    model.eval()
    for (bert_batch, graph_batch1, graph_batch2, targets) in eval_tqdm:
        optimizer.zero_grad()

        bert_batch = bert_batch.to(device)
        graph_batch1 = graph_batch1.to(device)
        graph_batch2 = graph_batch2.to(device)
        targets = targets.clone().detach().to(device)

        with torch.no_grad():
            logits = model(bert_batch, graph_batch1, graph_batch2)

        loss = loss_func(logits.view(-1, 2), targets.view(-1))

        pred_labels = torch.argmax(logits, dim=-1)
        true_labels = targets
        total_pred_labels.append(pred_labels)
        total_true_labels.append(true_labels)

        loss_sum += loss.item()

        eval_tqdm.set_postfix(loss=loss.item())
    return loss_sum / len(eval_dataloader), total_pred_labels, total_true_labels

In [ ]:
def test_loop():
    test_tqdm = tqdm(test_dataloader, unit="batch")
    test_tqdm.set_description(f'Epoch {epoch_counter}')
    loss_sum = 0
    total_pred_labels = []
    total_true_labels = []
    
    model.eval()
    for (bert_batch, graph_batch1, graph_batch2, targets) in test_tqdm:
        optimizer.zero_grad()

        bert_batch = bert_batch.to(device)
        graph_batch1 = graph_batch1.to(device)
        graph_batch2 = graph_batch2.to(device)
        targets = targets.clone().detach().to(device)

        with torch.no_grad():
            logits = model(bert_batch, graph_batch1, graph_batch2)

        loss = loss_func(logits.view(-1, 2), targets.view(-1))

        pred_labels = torch.argmax(logits, dim=-1)
        true_labels = targets
        total_pred_labels.append(pred_labels)
        total_true_labels.append(true_labels)

        loss_sum += loss.item()

        test_tqdm.set_postfix(loss=loss.item())
    return loss_sum / len(test_tqdm), total_pred_labels, total_true_labels

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, auc

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
n_iter = 0
valid_n_iter = 0
best_valid_loss = np.inf

for epoch_counter in range(num_epoch):
    loss, total_pred_labels, total_true_labels = train_loop()
    
    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

    wandb.log({"loss/train": loss})
    wandb.log({"accuracy/train": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/train": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/train": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/train": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/train": roc_auc_score(total_true_labels, total_pred_labels)})
    wandb.log({"roc_curve/train": roc_curve(total_true_labels, total_pred_labels)})


    loss, total_pred_labels, total_true_labels = eval_loop()

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

    wandb.log({"loss/eval": loss})
    wandb.log({"accuracy/validation": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/validation": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/validation": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/validation": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/validation": roc_auc_score(total_true_labels, total_pred_labels)})
    wandb.log({"roc_curve/validation": roc_curve(total_true_labels, total_pred_labels)})
    
    if loss < best_valid_loss:
        best_valid_loss = loss
        torch.save(model.state_dict(), os.path.join(log_dir, 'model_qsar_bbbp.pth'))
    
    if (epoch_counter + 1) % config['save_every_n_epochs'] == 0:
        torch.save(model.state_dict(), os.path.join(log_dir, 'model_qsar_bbbp_{}.pth'.format(str(epoch_counter))))

    if epoch_counter >= config['warm_up']:
                scheduler.step()


In [ ]:
loss, total_pred_labels, total_true_labels = test_loop()

total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

wandb.log({"roc_auc_score/test": roc_auc_score(total_true_labels, total_pred_labels)})


In [ ]:
loss, total_pred_labels, total_true_labels = eval_loop()

total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

wandb.log({"roc_auc_score/valid": roc_auc_score(total_true_labels, total_pred_labels)})


In [ ]:
roc_auc_score(total_true_labels, total_pred_labels, average='micro')

In [ ]:
wandb.finish()